# blinker

blinker使用:
    https://pythonhosted.org/blinker/

有点类似c++设计模式里的`中介者模式`

In [178]:
from blinker import *

In [179]:
s = Signal('s')

In [180]:
def callback(sender):
    print('callback from %s' %sender)

In [181]:
class A:
    pass

In [182]:
a = A()

In [183]:
s.connect(callback,sender=a)

<function __main__.callback(sender)>

同1个callback分2次，绑定到2个sender上;

In [184]:
b=A(); s.connect(callback,sender=b)

<function __main__.callback(sender)>

In [185]:
s.send()

[]

connect指定了sender 那么只有a发出的消息，才能被收到.

In [186]:
s.send(a)

callback from <__main__.A object at 0x7f372c3640f0>


[(<function __main__.callback(sender)>, None)]

In [187]:
s.send(b)

callback from <__main__.A object at 0x7f372c364278>


[(<function __main__.callback(sender)>, None)]

## 设计思路

Signal()相当于中间层，connect订阅了指定sender or (ANY)，并指定callback

```shm
1. Signal connect里肯定要维护所有注册到该signal上的callback(receivers);
2. Signal connect需要记录sender, 当send时，比对sender和send函数参数里指定的sender;
```

### 具体如何存储?

1. dict存储id:callback

In [188]:
s.receivers

{139874941697016: <weakref at 0x7f372c444b88; to 'function' at 0x7f372c367bf8 (callback)>}

另外用到了这种dict，相同的key,value将添加后list里？有点熟悉啊..leetcode实现过，叫hastdict.

In [189]:
from collections import defaultdict

In [190]:
S = defaultdict(set)

In [191]:
S

defaultdict(set, {})

In [192]:
S["1"].add(1);S["1"].add(2);S

defaultdict(set, {'1': {1, 2}})

`1个sender可能绑定多个callback,同样1个callback可以指定多个sender`,没有什么比这种结构更合适的了.


2. sender dict: 以sender对象为key, 以绑定callback id为 value;

In [193]:
s._by_sender


defaultdict(set,
            {139874941681904: {139874941697016},
             139874941682296: {139874941697016},
             0: set()})

3. receiver dict: 以callback为key,以指定的的sender为value

In [194]:
s._by_receiver

defaultdict(set, {139874941697016: {139874941681904, 139874941682296}})

### receiver_connected

是1个lazyproperty方式增加的属性，也是1个signal. connect调用后，将触发注册在该signal上的callback，比如下面的connected;

In [195]:
@s.receiver_connected.connect
def connected(*args, **kwargs):
    print('connected')

In [196]:
s.receiver_connected.receivers

{139874941697424: <weakref at 0x7f372c444768; to 'function' at 0x7f372c367d90 (connected)>}

connect后，触发receiver_connected的callback:

In [197]:
s.connect(callback,a)

connected


<function __main__.callback(sender)>

### reference

应该是blinker的复杂部分了。之前应该要了解`weakref`

比如在connect里:
```py
receiver_id = hashable_identity(receiver)
if weak:
    receiver_ref = reference(receiver, self._cleanup_receiver)
    receiver_ref.receiver_id = receiver_id
else:
    receiver_ref = receiver
```

reference： 要求object .即callback要有im_self, or__self__的属性.

annotation_xx理解为空，未实现;

```py

def reference(object, callback=None, **annotations):
    """Return an annotated weak ref."""
    if callable(object):
        weak = callable_reference(object, callback)
    else:
        weak = annotatable_weakref(object, callback)
    for key, value in annotations.items():
        setattr(weak, key, value)
    return weak


def callable_reference(object, callback=None):
    """Return an annotated weak ref, supporting bound instance methods."""
    if hasattr(object, 'im_self') and object.im_self is not None:
        return BoundMethodWeakref(target=object, on_delete=callback)
    elif hasattr(object, '__self__') and object.__self__ is not None:
        return BoundMethodWeakref(target=object, on_delete=callback)
    return annotatable_weakref(object, callback)
```

再看BoundMethodWeakref:

给BoundeMethod来做1个weakref,
什么是bound method?

In [207]:
class A:
    def f(self):
        pass 

<function __main__.A.f(self)>

unbound method

In [210]:
A.f 

<function __main__.A.f(self)>

bound method

In [211]:
a = A();a.f

<bound method A.f of <__main__.A object at 0x7f372c373470>>

1个method 可以bound到很多instance上.

In [212]:
b = A();
b.f

<bound method A.f of <__main__.A object at 0x7f372c373eb8>>

所以bound method是生命周期，可能和多个instance有关的,这就是下面`_all_instances`的意义

```py
class BoundMethodWeakref(object):
    """'Safe' and reusable weak references to instance methods.

    BoundMethodWeakref objects provide a mechanism for referencing a
    bound method without requiring that the method object itself
    (which is normally a transient object) is kept alive.  Instead,
    the BoundMethodWeakref object keeps weak references to both the
    object and the function which together define the instance method.

    Attributes:

    - ``key``: The identity key for the reference, calculated by the
      class's calculate_key method applied to the target instance method.

    - ``deletion_methods``: Sequence of callable objects taking single
      argument, a reference to this object which will be called when
      *either* the target object or target function is garbage
      collected (i.e. when this object becomes invalid).  These are
      specified as the on_delete parameters of safe_ref calls.

    - ``weak_self``: Weak reference to the target object.

    - ``weak_func``: Weak reference to the target function.

    Class Attributes:

    - ``_all_instances``: Class attribute pointing to all live
      BoundMethodWeakref objects indexed by the class's
      calculate_key(target) method applied to the target objects.
      This weak value dictionary is used to short-circuit creation so
      that multiple references to the same (object, function) pair
      produce the same BoundMethodWeakref instance.
    """

    _all_instances = weakref.WeakValueDictionary()

    def __new__(cls, target, on_delete=None, *arguments, **named):
        """Create new instance or return current instance.

        Basically this method of construction allows us to
        short-circuit creation of references to already- referenced
        instance methods.  The key corresponding to the target is
        calculated, and if there is already an existing reference,
        that is returned, with its deletion_methods attribute updated.
        Otherwise the new instance is created and registered in the
        table of already-referenced methods.
        """
        key = cls.calculate_key(target)
        current = cls._all_instances.get(key)
        if current is not None:
            current.deletion_methods.append(on_delete)
            return current
        else:
            base = super(BoundMethodWeakref, cls).__new__(cls)
            cls._all_instances[key] = base
            base.__init__(target, on_delete, *arguments, **named)
            return base

    def __init__(self, target, on_delete=None):
        """Return a weak-reference-like instance for a bound method.

        - ``target``: The instance-method target for the weak reference,
          must have im_self and im_func attributes and be
          reconstructable via the following, which is true of built-in
          instance methods::

            target.im_func.__get__( target.im_self )

        - ``on_delete``: Optional callback which will be called when
          this weak reference ceases to be valid (i.e. either the
          object or the function is garbage collected).  Should take a
          single argument, which will be passed a pointer to this
          object.
        """
        def remove(weak, self=self):
            """Set self.isDead to True when method or instance is destroyed."""
            methods = self.deletion_methods[:]
            del self.deletion_methods[:]
            try:
                del self.__class__._all_instances[self.key]
            except KeyError:
                pass
            for function in methods:
                try:
                    if callable(function):
                        function(self)
                except Exception:
                    try:
                        traceback.print_exc()
                    except AttributeError:
                        e = sys.exc_info()[1]
                        print ('Exception during saferef %s '
                               'cleanup function %s: %s' % (self, function, e))
        self.deletion_methods = [on_delete]
        self.key = self.calculate_key(target)
        im_self = get_self(target)
        im_func = get_func(target)
        self.weak_self = weakref.ref(im_self, remove)
        self.weak_func = weakref.ref(im_func, remove)
        self.self_name = str(im_self)
        self.func_name = str(im_func.__name__)

    def calculate_key(cls, target):
        """Calculate the reference key for this reference.

        Currently this is a two-tuple of the id()'s of the target
        object and the target function respectively.
        """
        return (id(get_self(target)), id(get_func(target)))
    calculate_key = classmethod(calculate_key)

    def __str__(self):
        """Give a friendly representation of the object."""
        return "%s(%s.%s)" % (
            self.__class__.__name__,
            self.self_name,
            self.func_name,
            )

    __repr__ = __str__

    def __nonzero__(self):
        """Whether we are still a valid reference."""
        return self() is not None

    def __cmp__(self, other):
        """Compare with another reference."""
        if not isinstance(other, self.__class__):
            return cmp(self.__class__, type(other))
        return cmp(self.key, other.key)

    def __call__(self):
        """Return a strong reference to the bound method.

        If the target cannot be retrieved, then will return None,
        otherwise returns a bound instance method for our object and
        function.

        Note: You may call this method any number of times, as it does
        not invalidate the reference.
        """
        target = self.weak_self()
        if target is not None:
            function = self.weak_func()
            if function is not None:
                return function.__get__(target)
        return Nonem
```